# Progetto HF-VRP-DL

In [3]:
import random
import math
from gurobipy import Model, GRB, quicksum

def generate_instance(num_ports=5, num_ships=2, seed=42):
    random.seed(seed)

    # Navi
    ships = []
    for i in range(num_ships):
        capacity = random.randint(80, 200)
        cost_per_unit = round(random.uniform(0.8, 1.5), 2)
        draft_empty = round(random.uniform(3.5, 6.0), 2)
        draft_per_unit = round(random.uniform(0.01, 0.03), 3)
        ships.append({
            'id': f'N{i+1}',
            'capacity': capacity,
            'cost_per_unit': cost_per_unit,
            'draft_empty': draft_empty,
            'draft_per_unit': draft_per_unit
        })

    # Porti
    ports = []
    for i in range(num_ports):
        x = random.randint(0, 100)
        y = random.randint(0, 100)
        demand = random.randint(10, 40)  # Ridotto il range della domanda per evitare infeasibility
        draft_limit = round(random.uniform(5.0, 8.0), 2)  # Aumentato il limite di pescaggio
        ports.append({
            'id': f'P{i+1}',
            'x': x,
            'y': y,
            'demand': demand,
            'draft_limit': draft_limit
        })

    # Distanze euclidee
    distances = {}
    for i in range(num_ports):
        for j in range(num_ports):
            if i != j:
                pi, pj = ports[i], ports[j]
                dist = math.sqrt((pi['x'] - pj['x'])**2 + (pi['y'] - pj['y'])**2)
                distances[(pi['id'], pj['id'])] = round(dist, 2)

    return {
        'ships': ships,
        'ports': ports,
        'distances': distances
    }

# Istanza di test con valori compatibili
ships = [
    {'id': 'N1', 'capacity': 120, 'cost_per_unit': 1.2, 'draft_empty': 4.0, 'draft_per_unit': 0.02},
    {'id': 'N2', 'capacity': 150, 'cost_per_unit': 1.0, 'draft_empty': 4.5, 'draft_per_unit': 0.015}
]

ports = [
    {'id': 'P1', 'x': 10, 'y': 20, 'demand': 30, 'draft_limit': 7.0},  # Deposito con limite alto
    {'id': 'P2', 'x': 15, 'y': 25, 'demand': 40, 'draft_limit': 6.5},
    {'id': 'P3', 'x': 20, 'y': 30, 'demand': 35, 'draft_limit': 6.0},
    {'id': 'P4', 'x': 25, 'y': 35, 'demand': 45, 'draft_limit': 5.5},
    {'id': 'P5', 'x': 30, 'y': 40, 'demand': 20, 'draft_limit': 5.0}
]

# Calcolo distanze
distances = {}
for i in ports:
    for j in ports:
        if i['id'] != j['id']:
            dist = math.sqrt((i['x'] - j['x'])**2 + (i['y'] - j['y'])**2)
            distances[(i['id'], j['id'])] = round(dist, 2)

# Parametri
port_ids = [p['id'] for p in ports]
depot = port_ids[0]  # P1 è il deposito
ship_ids = [s['id'] for s in ships]

demand = {p['id']: p['demand'] for p in ports}
draft_limit = {p['id']: p['draft_limit'] for p in ports}

capacity = {s['id']: s['capacity'] for s in ships}
cost = {s['id']: s['cost_per_unit'] for s in ships}
draft_empty = {s['id']: s['draft_empty'] for s in ships}
draft_per_unit = {s['id']: s['draft_per_unit'] for s in ships}

# Modello
m = Model("HF-VRP-DL")

# Variabili
x = m.addVars(port_ids, port_ids, ship_ids, vtype=GRB.BINARY, name="x")
y = m.addVars(port_ids, ship_ids, vtype=GRB.BINARY, name="y")
q = m.addVars(port_ids, ship_ids, vtype=GRB.CONTINUOUS, lb=0.0, name="q")
l = m.addVars(port_ids, ship_ids, vtype=GRB.CONTINUOUS, lb=0.0, name="l")

# Variabile ausiliaria per eliminazione subtour
u = m.addVars(port_ids, ship_ids, vtype=GRB.CONTINUOUS, lb=1.0, ub=len(port_ids), name="u")

# Obiettivo: minimizzare il costo totale
m.setObjective(
    quicksum(cost[k] * distances[i, j] * x[i, j, k]
             for i in port_ids for j in port_ids if i != j for k in ship_ids),
    GRB.MINIMIZE
)

# Vincoli

# (1) Ogni porto (tranne il deposito) è servito esattamente una volta
for i in port_ids:
    if i != depot:
        m.addConstr(quicksum(y[i, k] for k in ship_ids) == 1)

# (2) Capacità nave
for k in ship_ids:
    m.addConstr(quicksum(q[i, k] for i in port_ids if i != depot) <= capacity[k])

# (3) Domanda soddisfatta
for i in port_ids:
    if i != depot:
        m.addConstr(quicksum(q[i, k] for k in ship_ids) == demand[i])

# (4) Draft limit
for i in port_ids:
    for k in ship_ids:
        m.addConstr(draft_empty[k] + draft_per_unit[k] * l[i, k] <= draft_limit[i] + (1 - y[i, k]) * 1000)

# (5) Carico iniziale (porto P1 come punto di partenza)
for k in ship_ids:
    m.addConstr(l[depot, k] == quicksum(q[i, k] for i in port_ids if i != depot))

# (6) Aggiornamento carico residuo (corretto)
for k in ship_ids:
    for i in port_ids:
        for j in port_ids:
            if i != j:
                m.addConstr(
                    l[j, k] <= l[i, k] - q[j, k] + capacity[k] * (1 - x[i, j, k])
                )

# (7) Flusso bilanciato
for k in ship_ids:
    for i in port_ids:
        m.addConstr(
            quicksum(x[i, j, k] for j in port_ids if j != i) ==
            quicksum(x[j, i, k] for j in port_ids if j != i)
        )

# (8) Collegamento tra x e y
for i in port_ids:
    if i != depot:
        for k in ship_ids:
            m.addConstr(quicksum(x[i, j, k] for j in port_ids if j != i) == y[i, k])
            m.addConstr(quicksum(x[j, i, k] for j in port_ids if j != i) == y[i, k])

# (9) Ogni nave parte dal deposito
for k in ship_ids:
    m.addConstr(quicksum(x[depot, j, k] for j in port_ids if j != depot) <= 1)
    m.addConstr(quicksum(x[j, depot, k] for j in port_ids if j != depot) <= 1)

# (10) Eliminazione subtour (Miller-Tucker-Zemlin)
for i in port_ids:
    if i != depot:
        for j in port_ids:
            if j != depot and i != j:
                for k in ship_ids:
                    m.addConstr(u[i, k] - u[j, k] + len(port_ids) * x[i, j, k] <= len(port_ids) - 1)

# Risoluzione
m.optimize()

# Output
if m.status == GRB.OPTIMAL:
    print("\nSoluzione ottimale trovata!")
    print(f"Costo totale: {m.objVal:.2f}")
    
    for k in ship_ids:
        route = []
        load = 0
        
        # Verifica se la nave è utilizzata
        if sum(y[i, k].X for i in port_ids if i != depot) > 0:
            print(f"\nNave {k}:")
            
            # Trova il percorso
            current = depot
            while True:
                route.append(current)
                next_port = None
                for j in port_ids:
                    if j != current and x[current, j, k].X > 0.5:
                        next_port = j
                        break
                if next_port is None or next_port == depot and len(route) > 1:
                    route.append(depot)  # Ritorno al deposito
                    break
                current = next_port
            
            # Stampa percorso
            print(f"  Percorso: {' → '.join(route)}")
            
            # Stampa dettagli carico
            for i in port_ids:
                if i != depot and q[i, k].X > 0:
                    print(f"  Porto {i}: {q[i, k].X:.1f} unità, Carico residuo: {l[i, k].X:.1f}")
        else:
            print(f"\nNave {k}: non utilizzata")
            
else:
    print("\n⚠️ Modello infeasible. Controlla capacità, domanda e limiti di pescaggio.")
    m.computeIIS()
    m.write("model.ilp")
    print("Vincoli in conflitto salvati in 'model.ilp'")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 116 rows, 90 columns and 462 nonzeros
Model fingerprint: 0x02e8a404
Variable types: 30 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+03]
  Objective range  [7e+00, 3e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 82.0120000
Presolve removed 13 rows and 25 columns
Presolve time: 0.00s
Presolved: 103 rows, 65 columns, 574 nonzeros
Variable types: 21 continuous, 44 integer (44 binary)

Root relaxation: objective 2.941120e+01, 23 iterations, 0.00 seconds (0.00 work units)



    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   29.41120    0   12   82.01200   29.41120  64.1%     -    0s
     0     0   41.47733    0   12   82.01200   41.47733  49.4%     -    0s
H    0     0                      67.8720000   42.42000  37.5%     -    0s
     0     0   42.42000    0   10   67.87200   42.42000  37.5%     -    0s
     0     0   49.20720    0    8   67.87200   49.20720  27.5%     -    0s
     0     0   50.37880    0    8   67.87200   50.37880  25.8%     -    0s
     0     0   53.02500    0    6   67.87200   53.02500  21.9%     -    0s
     0     0   56.56000    0    6   67.87200   56.56000  16.7%     -    0s

Cutting planes:
  Gomory: 2
  Implied bound: 4
  Clique: 3
  MIR: 8
  RLT: 1

Explored 1 nodes (98 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 67.872 82.012 

Optimal solut